# COhPy May 2016 Challenge
> Andrew Kubera

# 1 -
Write a generator function that implements the [Python range
   function](https://docs.python.org/3/library/functions.html#func-range)
   (without using range()).


In [1]:
import operator

def Range(start, stop=None, step=1):
    """
    Get range generator.
    """
    # the start value is actually stop - swap them
    if stop is None:
        start, stop = 0, int(start)
    # ensure all variables are integers
    start, stop, step = map(int, (start, stop, step))
    
    if step < 0:
        cmp = operator.gt
    elif step > 0:
        cmp = operator.lt
    else:
        raise ValueError("Third argument must NOT be zero")

    i = start
    while cmp(i, stop):
        yield i
        i += step

In [2]:
Range(5)

<generator object Range at 0x10e813c50>

In [3]:
list(Range(0, 5)), list(Range(-5, 0))

([0, 1, 2, 3, 4], [-5, -4, -3, -2, -1])

In [4]:
list(Range(0, -5, -1))

[0, -1, -2, -3, -4]

In [5]:
list(Range(0, 10, 2)), list(Range(0, -10, -2))

([0, 2, 4, 6, 8], [0, -2, -4, -6, -8])

# 2 -
Write a generator function that generates prime numbers via the [Sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes).


In [6]:
import math

def sieve(n):
    # create list of n booleans indicating whether index
    # is prime. 0 and 1 are automatically not prime. The
    # rest are true
    res = [False, False] + [True] * (n - 2)
    
    # function returning iterator (filter) of all indexes
    # which are True
    get_true_items = lambda *args: filter(res.__getitem__, Range(*args))

    # Only have to loop through first √n items
    sqrt = int(math.sqrt(n))
    
    # Set all composite indices to False
    for i in get_true_items(sqrt + 1):
        yield i # the next 'true' item is prime
        for j in Range((n + 1) // i - i):
            res[i * (i + j)] = False

    # yield the rest of the true items
    yield from get_true_items(sqrt + 1, n)

In [7]:
list(sieve(20))

[2, 3, 5, 7, 11, 13, 17, 19]

In [8]:
list(sieve(200)) == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]

True

# 3 -
Write a generator function that given a string, generates all [permutations](https://en.wikipedia.org/wiki/Permutation) of that string.

### "Cheating" Solution
Use the itertools permutations function

In [9]:
import itertools
list(itertools.permutations("abc"))

[('a', 'b', 'c'),
 ('a', 'c', 'b'),
 ('b', 'a', 'c'),
 ('b', 'c', 'a'),
 ('c', 'a', 'b'),
 ('c', 'b', 'a')]

Does not return a string, let's write a generator to generate strings

In [10]:
def simple_permute(string):
    for permutation in itertools.permutations(string):
        yield ''.join(permutation)

In [11]:
list(simple_permute('abc'))

['abc', 'acb', 'bac', 'bca', 'cab', 'cba']

### Real Solution
Permute recursively!
* If len(string) - return string
* Otherwise, pop first letter off
* Get each permutation of substring
* yield each string generated by 'moving' the first letter through substring

In [12]:
def permute_string(string):
    if len(string) == 0:
        return
    elif len(string) == 1:
        yield string
    else:
        a = string[0]
        for p in permute_string(string[1:]):
            for i in range(len(p) + 1):
                yield "{}{}{}".format(p[:i], a, p[i:])

In [13]:
list(permute_string('a'))

['a']

In [14]:
list(permute_string('ab'))

['ab', 'ba']

In [15]:
list(permute_string('abc'))

['abc', 'bac', 'bca', 'acb', 'cab', 'cba']

In [16]:
len(list(permute_string('abcd'))) == math.factorial(4)

True

In [17]:
list(permute_string('aaa'))

['aaa', 'aaa', 'aaa', 'aaa', 'aaa', 'aaa']

# 4 -
Write a generator function that generates all [Pythagorean Triples](https://en.wikipedia.org/wiki/Pythagorean_triple) starting with (3,4,5).


### BRUTE FORCE METHOD

In [18]:

def p_triples(start=3, stop=100):
    for a in range(start, stop):
        for b in range(a + 1, stop):
            s = math.sqrt(a ** 2 + b ** 2)
            c = int(s)
            if s == c:
                # check if this number is factorable (ignore)
                x = math.gcd(a, b)
                if x != 1 and c / x == c // x:
                    continue

                yield (a, b, c)


In [19]:
list(filter(lambda x: x[2] < 100, p_triples(3, 250)))

[(3, 4, 5),
 (5, 12, 13),
 (7, 24, 25),
 (8, 15, 17),
 (9, 40, 41),
 (11, 60, 61),
 (12, 35, 37),
 (13, 84, 85),
 (16, 63, 65),
 (20, 21, 29),
 (28, 45, 53),
 (33, 56, 65),
 (36, 77, 85),
 (39, 80, 89),
 (48, 55, 73),
 (65, 72, 97)]

**Euclid's formula** tells us that for integers $m > n$:
$$a = m ^ 2 - n ^ 2, b = 2mn, c = m^2 + n ^2$$

In [20]:
def euclid_triples(stop=600):
    for n in range(1, stop):
        for m in range(n + 1, stop):
            a = m ** 2 - n ** 2
            b = 2 * m * n
            c =  m ** 2 + n ** 2
            x = math.gcd(a, b)
            if x != 1 and c / x == c // x:
                continue
            assert c == math.sqrt(a ** 2 + b ** 2)
            yield (a, b, c)

In [21]:
sorted(euclid_triples(10), key=lambda x: x[2])

[(3, 4, 5),
 (5, 12, 13),
 (15, 8, 17),
 (7, 24, 25),
 (21, 20, 29),
 (35, 12, 37),
 (9, 40, 41),
 (45, 28, 53),
 (11, 60, 61),
 (63, 16, 65),
 (33, 56, 65),
 (55, 48, 73),
 (77, 36, 85),
 (13, 84, 85),
 (39, 80, 89),
 (65, 72, 97),
 (15, 112, 113),
 (17, 144, 145)]